## Can I get Foldseek working and executing?

In [1]:
import os
import subprocess

In [3]:
EXEC_PATH = os.path.join("..", "exec", "foldseek", "bin", "foldseek")
print(EXEC_PATH)

../exec/foldseek/bin/foldseek


In [ ]:
def bash_cmd(cmd: str | list[str]) -> str:
    return subprocess.check_output(cmd, shell=True).decode()

In [21]:
def parse_easy_search_output(f) -> list[list]:
    lines = f.readlines()

    parsed_lines = []
    for line in lines:
        parsed_line = []
        for column in line.strip("\n").split("\t"):
            try:
                column = float(column)
            except ValueError:
                pass
            parsed_line.append(column)
        parsed_lines.append(parsed_line)

    return parsed_lines

In [14]:
class Executable:
	def __init__(self, path):
		if not os.path.exists(path):
			raise ImportError(f"Could not find {path}")
		self.path = path
	def __call__(self, cmd=""):
		return subprocess.check_output(f"{self.path} {cmd}", shell=True).decode()
	def __repr__(self):
		return f"Executable(path='{self.path}')"


foldseek = Executable(EXEC_PATH)

query = os.path.join("..", "data", "venome")
target = query # just search itself to test result 
temp_dir = os.path.join(".", "temp")
out_file = os.path.join(temp_dir, "output")
flags = "--format-output query,target,prob"
foldseek(f"easy-search {query} {target} {out_file} {temp_dir} {flags}")

"Create directory ./temp\neasy-search ../data/venome ../data/venome ./temp/output ./temp --format-output query,target,prob \n\nMMseqs Version:              \tb7c58acb1201d7932e945bb20e49906fb691437d\nSeq. id. threshold           \t0\nCoverage threshold           \t0\nCoverage mode                \t0\nMax reject                   \t2147483647\nMax accept                   \t2147483647\nAdd backtrace                \tfalse\nTMscore threshold            \t0\nTMscore threshold mode       \t0\nTMalign hit order            \t0\nTMalign fast                 \t1\nPreload mode                 \t0\nThreads                      \t12\nVerbosity                    \t3\nLDDT threshold               \t0\nSort by structure bit score  \t1\nAlignment type               \t2\nExact TMscore                \t0\nSubstitution matrix          \taa:3di.out,nucl:3di.out\nAlignment mode               \t3\nAlignment mode               \t0\nE-value threshold            \t10\nMin alignment length         \t0\nSeq. i

In [16]:
# works!
with open(out_file, "r") as f:
	print(f.read())

Gh_comp2027_c0_seq1	Gh_comp2027_c0_seq1	1.000
Gh_comp2027_c0_seq1	Gh_comp1373_c0_seq1	0.000
Gh_comp1373_c0_seq1	Gh_comp1373_c0_seq1	1.000
Gh_comp1373_c0_seq1	Gh_comp2027_c0_seq1	0.000



In [26]:
import shutil

class Foldseek(Executable):
    def __init__(self):
        super().__init__(EXEC_PATH)
        self.temp_dir = os.path.join(".", "temp")
        self.out_file = os.path.join(self.temp_dir, "output")

    def _delete_temp(self):
        shutil.rmtree(self.temp_dir)
        
    def _parse_out_file(self):
        with open(self.out_file, "r") as f:
            lines = f.readlines()
            parsed_lines = []
            for line in lines:
                parsed_line = []
                for column in line.strip("\n").split("\t"):
                    try:
                        column = float(column)
                    except ValueError:
                        pass
                    parsed_line.append(column)
                parsed_lines.append(parsed_line)

            return parsed_lines

    def search(self, query, target, flags = "--format-output query,target,prob", clean=True):
        stdout = self(f"easy-search {query} {target} {self.out_file} {self.temp_dir} {flags}")
        search_out = self._parse_out_file()

        if clean:
            self._delete_temp()

        return search_out, stdout


fs = Foldseek()
print(fs.search(query, query))

([['Gh_comp1373_c0_seq1', 'Gh_comp1373_c0_seq1', 1.0, 332.0, 0.0, 0.0, 1.0, 332.0, 1.0, 332.0, 3.405e-71, 2888.0], ['Gh_comp1373_c0_seq1', 'Gh_comp2027_c0_seq1', 0.125, 77.0, 59.0, 0.0, 80.0, 147.0, 81.0, 157.0, 3.592, 2.0], ['Gh_comp2027_c0_seq1', 'Gh_comp2027_c0_seq1', 1.0, 286.0, 0.0, 0.0, 1.0, 286.0, 1.0, 286.0, 9.106e-54, 2543.0], ['Gh_comp2027_c0_seq1', 'Gh_comp1373_c0_seq1', 0.125, 77.0, 59.0, 0.0, 81.0, 157.0, 80.0, 147.0, 5.152, 0.0]], "./temp/output exists and will be overwritten\neasy-search ../data/venome ../data/venome ./temp/output ./temp \n\nMMseqs Version:              \tb7c58acb1201d7932e945bb20e49906fb691437d\nSeq. id. threshold           \t0\nCoverage threshold           \t0\nCoverage mode                \t0\nMax reject                   \t2147483647\nMax accept                   \t2147483647\nAdd backtrace                \tfalse\nTMscore threshold            \t0\nTMscore threshold mode       \t0\nTMalign hit order            \t0\nTMalign fast                 \t1\nPr